In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot  as plt
import re
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from sklearn.utils import resample

In [ ]:
# Reading Excel files
df_ego = pd.read_excel('Pre-treatment_all.xlsx', sheet_name='EGO_planner')
df_kino = pd.read_excel('Pre-treatment_all.xlsx', sheet_name='Fast_planner')
df_topo = pd.read_excel('Pre-treatment_all.xlsx', sheet_name='PGO')
df_navRL = pd.read_excel('Pre-treatment_all.xlsx', sheet_name='NavRL')
df_straight = pd.read_excel('Pre-treatment_all.xlsx', sheet_name='Straight')


In [ ]:
# Creating data structures: the success of algorithms in seven scenarios
data = {
    'Forest': {
        'EGO-Planner': df_ego['Forest'],
        'Fast-Planner': df_kino['Forest'],
        'PGO': df_topo['Forest'],
        'NavRL': df_navRL['Forest'],
        'Straight': df_straight['Forest'],
    },
    'Urban': {
        'EGO-Planner': df_ego['Urban'],
        'Fast-Planner': df_kino['Urban'],
        'PGO': df_topo['Urban'],
        'NavRL': df_navRL['Urban'],
        'Straight': df_straight['Urban'],
    },
    'Random-Angle Cylinder': {
        'EGO-Planner': df_ego['Random-Angle'],
        'Fast-Planner': df_kino['Random-Angle'],
        'PGO': df_topo['Random-Angle'],
        'NavRL': df_navRL['Random-Angle'],
        'Straight': df_straight['Random-Angle'],
    },
    'Narrow-Gap': {
        'EGO-Planner': df_ego['Narrow'],
        'Fast-Planner': df_kino['Narrow'],
        'PGO': df_topo['Narrow'],
        'NavRL': df_navRL['Narrow'],
        'Straight': df_straight['Narrow'],
    },
    'Sudden-Drop': {
        'EGO-Planner': df_ego['Sudden‑Drop'],
        'Fast-Planner': df_kino['Sudden‑Drop'],
        'PGO': df_topo['Sudden‑Drop'],
        'NavRL': df_navRL['Sudden‑Drop'],
        'Straight': df_straight['Sudden‑Drop'],
    },
    'Maze': {
        'EGO-Planner': df_ego['Maze'],
        'Fast-Planner': df_kino['Maze'],
        'PGO': df_topo['Maze'],
        'NavRL': df_navRL['Maze'],
        'Straight': df_straight['Maze'],
    },
    'Random Perlin-Noise': {
        'EGO-Planner': df_ego['Perlin-Noise'],
        'Fast-Planner': df_kino['Perlin-Noise'],
        'PGO': df_topo['Perlin-Noise'],
        'NavRL': df_navRL['Perlin-Noise'],
        'Straight': df_straight['Perlin-Noise'],
    },
}

print(data['Urban']['PGO'])

In [ ]:
# Functions to calculate Bootstrap confidence intervals
def bootstrap_ci(data, n_bootstraps=5000, ci=95):
    means = []
    for _ in range(n_bootstraps):
        sample = resample(data, replace=True)
        means.append(np.mean(sample))
    
    mean_val = np.mean(data)
    alpha = (100 - ci) / 2
    lower = np.percentile(means, alpha)
    upper = np.percentile(means, 100 - alpha)
    
    return mean_val, mean_val - lower, upper - mean_val


In [ ]:
# Preparation of plotting data
algorithms = ['EGO-Planner', 'Fast-Planner', 'PGO', 'NavRL', 'Straight']
scenes = ['Forest', 'Urban', 'Random-Angle Cylinder', 'Narrow-Gap', 'Sudden-Drop', 'Maze', 'Random Perlin-Noise']

# Creating Resulting Data Structures
results = {algo: {} for algo in algorithms}
for scene in scenes:
    for algo in algorithms:
        mean, lower_err, upper_err = bootstrap_ci(data[scene][algo])
        results[algo][scene] = {
            'mean': mean,
            'lower_err': lower_err,
            'upper_err': upper_err
        }

In [ ]:
# Convert to DataFrame format
df_means = pd.DataFrame(
    [[results[algo][scene]['mean'] for scene in scenes] for algo in algorithms],
    index=algorithms,
    columns=scenes
)
# Creating an error data frame (the corrected and corrected way)
df_errors = pd.DataFrame(
    [[[results[algo][scene]['lower_err'], results[algo][scene]['upper_err']] 
    for scene in scenes] 
    for algo in algorithms],
    index=algorithms,
    columns=scenes
)
print(df_means)
print(df_errors)

In [ ]:
# Setting up a professional academic chart style
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Arial', 'DejaVu Sans Mono','Times New Roman'],
    'font.size': 10,
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.dpi': 800,
    'figure.figsize': (16, 10),
    'savefig.dpi': 800,
    'savefig.bbox': 'tight',
    'savefig.transparent': True,
})

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for ax in axes:
    ax.set_aspect('auto')

algorithm_colors = {
    'EGO-Planner': "#CCDCFF",
    'Fast-Planner': '#CCDCFF',
    'PGO': '#CCDCFF',
    'NavRL': '#CCDCFF',
    'Straight': '#CCDCFF'
}

plot_indices = [0, 1, 2, 4, 5, 6, 7]

empty_axis_index = 3

# Cyclic drawing of each scene
for i, scene in enumerate(scenes):
    if i >= len(plot_indices):
        break
        
    ax_index = plot_indices[i]
    ax = axes[ax_index]
    
    mean_data = df_means[scene]
    ci_data = df_errors[scene]
    
    means = mean_data.values * 100
    errors = []
    for algo in algorithms:
        lower_err = ci_data[algo][0] * 100
        upper_err = ci_data[algo][1] * 100
        errors.append([lower_err, upper_err])
    
    errors = np.array(errors).T
    
    x_pos = np.arange(len(algorithms))
    bar_width = 0.6
    
    for j, algo in enumerate(algorithms):
        ax.bar(
            x_pos[j],
            means[j],
            width=bar_width,
            color=algorithm_colors[algo],
            edgecolor='#2F528C',
            linewidth=0.8,
            alpha=0.9,
            label=algo if i == 0 else None
        )
    
    # Add error lines (Bootstrap confidence intervals)
    ax.errorbar(
        x_pos,
        means,
        yerr=errors,
        fmt='none',
        ecolor='#2F528C',
        elinewidth=1.2,
        capsize=4,
        capthick=1.2,
        alpha=0.8
    )

    ax.set_title(f'{scene}', fontsize=16, pad=10)

    if i == 0 or i == 3:
        ax.set_ylabel('Success Rate (%)', fontsize=12, labelpad=8)
    
    ax.set_xticks(x_pos)
    ax.set_xticklabels(algorithms, rotation=15, fontsize=9)

    ax.set_ylim(0, 100)

    ax.grid(axis='y', linestyle='--', alpha=0.2)
    
    for y in [20, 40, 60, 80, 100]:
        ax.axhline(y, color='gray', linestyle='-', alpha=0.1, zorder=0)

axes[empty_axis_index].axis('off')

plt.tight_layout()
plt.subplots_adjust(bottom=0.1, top=0.93, hspace=0.35, wspace=0.25)

plt.savefig('all_environments_performance.png', dpi=800)
plt.savefig('all_environments_performance.pdf', dpi=800)
plt.show()